<a href="https://colab.research.google.com/github/Shumin-li-mcit/CIS5300/blob/main/SL_CIS_5300_Homework_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 7: Fine-tuning Pre-trained Language Models

Due Date: July 17th, 2023 11:59PM ET

Total Points: 100 points

- **Overview**: In this assignment, we will explore how to take pre-trained language models and adapt them to do different natural language tasks. We'll cover:
   - Applying models to down-stream tasks without extra training (15 points)
   - Fine-tuning for sequence classification (20 points)
   - Fine-tuning for sequence pair classification (25 points)
   - Fine-tuning for token classification (40 points)
- Deliverables: This assignment has several deliverables:
   - Code (this notebook) (Automatic Graded)
- Grading: We will use the auto-grading system called PennGrader. To complete the homework assignment, you should implement anything marked with #TODO and run the cell with #PennGrader note.

Recommended Readings
- [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805) Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova. NAACL 2019.
- [Generalized Language Models](https://lilianweng.github.io/posts/2019-01-31-lm/) Lillian Weng. 2019
- [The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](http://jalammar.github.io/illustrated-bert/) Jay Alammar. 2019


## To get started, make a copy of this colab notebook into your google drive!

Also make sure you're using the GPU by going to `Runtime` -> `Change Runtime Type` and making sure that `Hardware Accelerator` is set to `GPU`. This will save you significant amounts of time when training models

In [1]:
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [2]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install penngrader-client

In [3]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [4]:
!cat notebook-config.yaml


grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'


In [5]:
from penngrader.grader import *

## TODO - Start
STUDENT_ID = 65847590 # YOUR PENN-ID GOES HERE AS AN INTEGER#
## TODO - End

SECRET = STUDENT_ID
grader = PennGrader('notebook-config.yaml', 'CIS5300_OL_23Su_HW7', STUDENT_ID, SECRET)

PennGrader initialized with Student ID: 65847590

Make sure this correct or we will not be able to store your grade


In [6]:
# check if the PennGrader is set up correctly
# do not change this cell, see if you get 4/4!
name_str = 'Shumin Li'
grader.grade(test_case_id = 'name_test', answer = name_str)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [7]:
# Comment this out if you want to enable huggingface warnings
import logging
logging.disable(logging.WARNING)

# This fixes colab's default encoding to match huggingface accelerate
import locale
locale.getpreferredencoding = lambda x=False: "UTF-8"

# Introduction: HuggingFace Transformers

In this homework we will be making use of the [HuggingFace](https://huggingface.co/) transformers library. This library provides support for downloading, running, and training language models and is an essential tool for professionals that want to deploy NLP technology.

We will be installing four huggingface libraries:

1. [Transformers](https://huggingface.co/docs/transformers/index) (Model inference): `!pip install transformers`
2. [Accelerate](https://huggingface.co/docs/accelerate/index) (Model training): `!pip install accelerate`
3. [Datasets](https://huggingface.co/docs/datasets/index) (Data processing): `!pip install datasets`
4. [Evaluate](https://huggingface.co/docs/evaluate/index) (Model evaluation): `!pip install evaluate`

We highly recommend that you reference the documentation linked above frequently as it will be a very valuable resource throughout this homework

In [8]:
%%capture
!pip install datasets
!pip install transformers==4.28.0
!pip install --upgrade accelerate
!pip install evaluate

In [9]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import datasets
from datasets import load_dataset, list_datasets
from evaluate import evaluator
import evaluate
import numpy as np
import torch
import copy

from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_id = 0 if device == 'cuda' else -1

# Section 1: Zero-Shot Application of Pre-Trained Language Models (15 points)

In this homework we'll be exploring different ways of using pre-trained language models to accomplish natural language tasks. We will be using BERT as our pre-trained language model for this homework.

BERT was introduced in late 2018 by the landmark paper [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805). It was trained via "Masked Language Modeling" a procedure that involves training the model to fill in [MASK] tokens in input sequences.

<!-- We will be using the "base" size BERT model (110 million parameters) and we will use the "cased" version (i.e. the model is trained to distinguish between upper and lower case letters).  -->

As you can see below, according to BERT, there is an 84.7% chance that the masked word is "Italian".

<!-- Note: We are running BERT via a [HuggingFace Pipeline](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/pipelines) object. Here we are using a `fill-mask` pipeline but many others exist and we will be using them extensively in this assignment. While this is not the only way to query a model with HuggingFace, it is the most convenient. -->

In [10]:
# Download and query the bert-base-cased model
bert_model = pipeline('fill-mask', model='bert-base-cased', device=device_id)
print(bert_model("I went to an [MASK] restaurant and ordered pasta."))

[{'score': 0.8473346829414368, 'token': 2169, 'token_str': 'Italian', 'sequence': 'I went to an Italian restaurant and ordered pasta.'}, {'score': 0.03583133965730667, 'token': 1890, 'token_str': 'Indian', 'sequence': 'I went to an Indian restaurant and ordered pasta.'}, {'score': 0.016542594879865646, 'token': 1385, 'token_str': 'old', 'sequence': 'I went to an old restaurant and ordered pasta.'}, {'score': 0.006616983562707901, 'token': 3427, 'token_str': 'empty', 'sequence': 'I went to an empty restaurant and ordered pasta.'}, {'score': 0.0058785974979400635, 'token': 6210, 'token_str': 'Egyptian', 'sequence': 'I went to an Egyptian restaurant and ordered pasta.'}]


In *theory* we should be able to use BERT to do natural language tasks such as Question Answering, Language Identification, Part-of-Speech Tagging, and even Translation by formulating our task in the style of a fill-in-the-blank sentence.

In [11]:
# Language Identification
print(bert_model("I am currently speaking in the [MASK] language.")[0])

# Factual QA
print(bert_model("The Declaration of Independence was written in the year [MASK].")[0])

# Part of speech tagging
print(bert_model("The word run is a [MASK].")[0])

# Translation
print(bert_model("The French word amour translates to [MASK] in English.")[0])

{'score': 0.18785126507282257, 'token': 1483, 'token_str': 'English', 'sequence': 'I am currently speaking in the English language.'}
{'score': 0.10682430118322372, 'token': 14447, 'token_str': '1776', 'sequence': 'The Declaration of Independence was written in the year 1776.'}
{'score': 0.2285984307527542, 'token': 12464, 'token_str': 'verb', 'sequence': 'The word run is a verb.'}
{'score': 0.05516679584980011, 'token': 1567, 'token_str': 'love', 'sequence': 'The French word amour translates to love in English.'}


However, it turns out that BERT really isn't very good at doing these tasks without extra training (as you can see below). In Section 1 of this homework we'll evaluate BERT without extra training on sentiment analysis to get an idea of where the base model is at. Then, in the next sections, we'll train the model and see how much we can improve the performance.

In [12]:
# Language Identification (failed)
print(bert_model("今[MASK]で話しています")[0])

# Factual QA (failed)
print(bert_model("The U.S.A. was founded in the year [MASK].")[0])

# Part of speech tagging (failed)
print(bert_model("The word golf is a [MASK].")[0])

# Translation (failed)
print(bert_model("The French word bonjour translates to [MASK] in English.")[0])

{'score': 0.5789173245429993, 'token': 100, 'token_str': '[UNK]', 'sequence': 'しています'}
{'score': 0.1736524999141693, 'token': 1196, 'token_str': 'before', 'sequence': 'The U. S. A. was founded in the year before.'}
{'score': 0.15804290771484375, 'token': 8155, 'token_str': 'joke', 'sequence': 'The word golf is a joke.'}
{'score': 0.016320636495947838, 'token': 6164, 'token_str': 'wolf', 'sequence': 'The French word bonjour translates to wolf in English.'}


### **Sequence Classification with pre-trained BERT**

In this section we'll be evaluating BERT for sentiment analysis without fine-tuning. This is purely for the purposes of demonstration as you'll be able to see the difference between what BERT does before and after fine-tuning.

In order to do a sequence classification task without fine-tuning BERT you need two things. You need
1. A mask-filling template to attach on to the end of the sequence
2. A set of `targets` which consist of the class labels.

Below you'll see an example of this procedure applied to sentiment analysis. For the sentence "I just saw a movie today and it was really great", we see that BERT outputs 2.78% chance of the `[MASK]` token being "positive" and 1.65% chance of it being "negative", thus we give the example a positive label.

In [13]:
bert_model("I just saw a movie today and it was really great. My opinion of the movie is [MASK].", targets=["positive", "negative"])

[{'score': 0.027819789946079254,
  'token': 3112,
  'token_str': 'positive',
  'sequence': 'I just saw a movie today and it was really great. My opinion of the movie is positive.'},
 {'score': 0.016482586041092873,
  'token': 4366,
  'token_str': 'negative',
  'sequence': 'I just saw a movie today and it was really great. My opinion of the movie is negative.'}]

Let's test BERT's ability to recognize sentiment using the [Yelp Reviews](https://huggingface.co/datasets/yelp_review_full) dataset. This dataset consists of 650,000 reviews with their user-assigned star ratings. The task is to determine how many stars (1 to 5) the user gave given the text of their review.

In [14]:
# Download the yelp dataset from huggingface
dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
dataset['test']['label'][0]

0

### **Dataset Processing**

There are many functions we can use to process a HuggingFace dataset object. One of the most useful is the [Map](https://huggingface.co/docs/datasets/process#map) function. This function applies some function `f` to every row of the dataset where `f` takes in a row of the dataset and returns a dictionary containing the new columns of the data (or any edits to existing columns). For example, given a dataset with a column `binary_label` that contains a binary label (0 or 1) you can create a new column with the opposite label by writing either one of the following functions:
```
def reverse_label(row):
  return {'reverse_label': int(not row["binary_label"])}

def reverse_label(row):
  row['reverse_label'] = int(not row[''binary_label"])
  return row
```
and then calling map on the dataset with the function
```
new_data = dataset.map(reverse_label)
```

### **Concatenating the Mask-Fill Template** (5 points)
Your first task will be to concatenate the mask-fill template onto every item in the `text` column of the dataset and add the output to the dataset as a new column named `input`.

Refer back to the documentation for the [Map](https://huggingface.co/docs/datasets/process#map) function if you're stuck

In [16]:
def concatenate_mask(row):
  # TODO: Concatenate the template to each example and return in a new column "input"
  template = " I give it a score of [MASK] out of 5."
  row["input"] = row["text"] + template
  return row

test_data = dataset['test'].map(concatenate_mask)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [17]:
grader.grade(test_case_id = 'test_concat_mask', answer = concatenate_mask)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### **Intro to the BERT Tokenizer**
Before taking in a sequence of text, language models need to break up the sequence into tokens. Since BERT uses the [Transformer](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model) architecture, it has a fixed maximum length of tokens it can process at any one time (that being 512). If you pass in a sequence of text longer than 512 tokens, BERT will throw an error:

In [18]:
# Giving BERT a piece of text with more than 512 tokens produces an error
try:
  bert_model("Hello " * 512 + " my name is [MASK]")
except RuntimeError as e:
  print(e)

The size of tensor a (518) must match the size of tensor b (512) at non-singleton dimension 1


### **Filtering the data for BERT's max length** (5 points)
Below we have provided code to load the BERT tokenizer. To run the tokenizer you can call it on a piece of text like `tokenizer("Hello students")`. This will return a dictionary with three arrays, `input_ids`, `attention_mask` and `token_type_ids`. The length of all three arrays should be equal to the total number of tokens in the sequence.

Since we need to make sure that no example in our input is longer than 512 tokens, your assignment now is to filter the dataset to only include examples that have 512 or less tokens. You can do this using the [Filter](https://huggingface.co/docs/datasets/process#select-and-filter) function. This function is similar to Map but returns a boolean where `True` indicates we should keep the row and `False` indicates we should remove it.

NOTE: The maximum sequence length of any model can be found through the `tokenizer.model_max_length` variable. If you dislike magic numbers in your code, you can use this instead of hard-coding 512.

In [19]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def filter_for_max_length(row):
  # TODO: Return True if the length of the input (in tokens) is less than or equal
  #        to the maximum length allowed by the BERT model
  tokens = tokenizer(row["input"])
  if len(tokens[0]) <= tokenizer.model_max_length:
    return True
  return False

filtered_data = test_data.filter(filter_for_max_length)

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [20]:
tokenizer(test_data["input"][0])

{'input_ids': [101, 146, 1400, 112, 1207, 112, 14337, 1121, 1172, 1105, 1439, 1160, 2277, 1400, 170, 3596, 119, 146, 1261, 1139, 1610, 1106, 170, 1469, 19459, 1106, 1267, 1191, 178, 1180, 1243, 1103, 4569, 10085, 1174, 117, 1133, 1152, 1163, 1103, 2255, 146, 1125, 170, 3596, 1108, 1272, 1103, 2166, 10085, 1125, 10816, 118, 22751, 12150, 117, 160, 11612, 1942, 136, 146, 1198, 1400, 1103, 15269, 1105, 1309, 1834, 1106, 1138, 1122, 10085, 1174, 136, 1188, 1108, 3155, 1106, 1129, 170, 1207, 15269, 119, 165, 183, 2240, 1261, 1103, 15269, 1166, 1106, 13580, 112, 188, 1105, 1152, 1500, 1143, 1115, 1800, 23609, 26405, 23132, 1139, 15269, 117, 1173, 1793, 1106, 10085, 1122, 119, 1573, 1175, 1132, 1231, 27408, 2365, 15269, 26632, 1468, 136, 146, 1525, 1115, 1304, 9803, 119, 1258, 8995, 1114, 1103, 2564, 1105, 3344, 1140, 1115, 1117, 8738, 1108, 1677, 21616, 1174, 1119, 1163, 1119, 112, 173, 1660, 1143, 170, 1207, 15269, 165, 107, 1142, 1159, 165, 107, 119, 165, 183, 2240, 1209, 1309, 1301, 1171,

In [21]:
grader.grade(test_case_id = 'test_filtered_length', answer = len(filtered_data))

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


Here we'll be randomly selecting 100 examples from our filtered dataset for testing purposes. We'll be doing this throughout this assignment both for training and testing to ensure runtime stays fast. In the real world, given unlimited time, we would ideally like to use the full datasets.

In [22]:
# Randomly sample 100 examples from the dataset (do not change!)
sampled_data = filtered_data.shuffle(seed=42).select(range(100))

### **Apply BERT to each example** (5 points)
As your final task for this section you must write a function that runs the BERT model on every sequence of text in the dataset's `input` column and outputs the most likely integer star rating (1-5) as predicted by BERT in a new column named `score`.

Refer back to the earlier parts of this Section for how to use BERT and make sure to use the `targets` field to specify the five possible star ratings `[1, 2, 3, 4, 5]`.

In [23]:
def apply_bert(row):
  # TODO: Compute the probabilities for each score and return the value of the most likely score
  prediction = bert_model(row['input'], targets=['1', '2', '3', '4', '5'])
  row['score'] = int(prediction[0]['token_str'])
  return row

predicted_data = sampled_data.map(apply_bert)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [24]:
bert_model(sampled_data['input'][0], targets=['1', '2', '3', '4', '5'])

[{'score': 0.5400611758232117,
  'token': 125,
  'token_str': '4',
  'sequence': "Probably has the best ddukboki on campus! It's made in such a way that you still want to pay for it'cause even though you can make your own version at home, their version is that much better. Also, they get the rice right every time ( it's the simple things that add up ). I give it a score of 4 out of 5."},
 {'score': 0.19706812500953674,
  'token': 124,
  'token_str': '3',
  'sequence': "Probably has the best ddukboki on campus! It's made in such a way that you still want to pay for it'cause even though you can make your own version at home, their version is that much better. Also, they get the rice right every time ( it's the simple things that add up ). I give it a score of 3 out of 5."},
 {'score': 0.1322878748178482,
  'token': 126,
  'token_str': '5',
  'sequence': "Probably has the best ddukboki on campus! It's made in such a way that you still want to pay for it'cause even though you can make your

In [25]:
grader.grade(test_case_id = 'test_predicted_data', answer = predicted_data['score'])

Correct! You earned 5.0/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### **Evaluating the Model**

Below we've written code to evaluate the model's accuracy. If you've done everything correctly you should get somewhere around 24% accuracy. Given that this task is five-way classification, this is only *barely* better than random chance! Clearly BERT cannot do sentiment analysis without extra training. Let's see if fine-tuning will help.



In [26]:
def accuracy(outputs, reference):
  return sum([o == r for o, r in zip(outputs, reference)]) / len(reference)

print(accuracy(predicted_data["score"], predicted_data["label"]))

0.24


# Section 2: Training BERT for Sentiment Analysis (20 points)

In this section we'll be training BERT for the same Sentiment Analysis task from Section 1. We'll start by giving a short introduction to the concept of fine-tuning.

### **What is fine-tuning?**

Fine-tuning is the process of further training a pre-trained model to accomplish a particular down-stream task. This is typically done by swapping out the pre-trained model's last layer (its head) for a new randomly initialized head and training both model and head.

### **Model "heads"**

The final output layer of a language model is typically called a "head". The standard head used by models when pre-training is called a "languge modeling head". This is a dense linear layer that projects the $D_{enc}$ length encodings of each of the $L_{context}$ tokens to a probability distribution over the vocabulary. The total size of this layer is thus ($L_{context} * D_{enc}$ x $|V|$).
<!-- The head and the network are always trained *together*. We compute the cross-entropy loss $L = -\log(P(w))$ using the output of the head and backpropagate through the head to the rest of the network. -->

### **Adding on a "Classification head"**

In Section 1 we used BERT's language modeling head to perform the Sentiment Analysis task. While we *can* train the model with its original LM head, this is unnecessary, as the LM head outputs probabilities over all tokens rather than just our targets. Thus, in this section we're going to remove the language modeling head and replace it with a classification head.

A classification head is very similar to a language modeling head. It is a dense linear layer that projects the $D_{enc}$ length encodings of each of the $L_{context}$ tokens to a probability distribution over the *$n$ classes* rather than the whole vocabulary. The total size of this layer is thus ($L_{context} * D_{enc}$ x $n$). This makes it significantly more efficient to train.

### **Using HuggingFace to swap heads**

Loading a model with a particular head is easy using HuggingFace. All you do is load the model as an instance of a given class. The classes we're using in this homework are as follows:
1. [AutoModelForMaskedLM](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForMaskedLM) - Language modeling Head
2. [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForSequenceClassification) - Classification Head
3. [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForTokenClassification) - Token Classification Head

Feel free to read the [documentation](https://huggingface.co/docs/transformers/model_doc/auto#auto-classes) for a list of other possible heads

In [27]:
# Load in the bert-base-cased model with a classification head (num_labels = number of classes)
classification_head_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

### **Training the Model + Classification head**

Let's now train the model + classification head for sentiment analysis. We will use the HuggingFace [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) class. We need to:
1. Tokenize and process the dataset
2. Create a `compute_metrics` function
3. Initialize the `Trainer` and specify the `TrainingArguments`

### **Step 1: Tokenize and Process the dataset** (5 points)

For this section you must implement the `tokenize_function`. This function should run the BERT tokenizer on every example in the dataset and add the output fields `input_ids`, `attention_mask`, and `token_type_ids` as new columns in the dataset.

This function should also also pad all sequences of less than 512 tokens to be exactly 512 tokens using the special `[PAD]` token and truncate sequences longer than 512 to be exactly 512. This is to help GPU parallelization and can be done by specifying certain arguments to the tokenizer (check the [documentation](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__)!)

In [28]:
# Step 1: Tokenize and Process the dataset
dataset = load_dataset("yelp_review_full")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(row):
  # TODO: Tokenize the dataset (Remember to pad and truncate to max length)
  tokens = tokenizer(row['text'], max_length = tokenizer.model_max_length, padding = 'max_length', truncation = True)
  return {'input_ids': tokens['input_ids'], 'token_type_ids': tokens['token_type_ids'], 'attention_mask': tokens['attention_mask']}

# Randomly select 1000 examples from the train and test data and tokenize - do not change!
# (Note: We are subsampling our data just so that training doesn't take too long)
train_data = dataset["train"].shuffle(seed=42).select(range(1000)).map(tokenize_function)
eval_data = dataset["test"].shuffle(seed=42).select(range(1000)).map(tokenize_function)

  0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [29]:
grader.grade(test_case_id = 'test_tokenized_data', answer = train_data[:500])

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### **Step 2: Define a `compute_metrics` function** (10 points)

For this function you'll be calculating accuracy between the predictions and true labels. The `probabilities` variable is a 2D numpy array of size ($n$ x $5$) containing probabilities for the 5 class labels for each of the $n$ examples in the input dataset.

You must write code to select the index with the highest probability for each row in the `probabilities` array then calculate the accuracy of the model with respect to the ground truth label. (HINT: the [np.argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) function is really nice for this)

In [30]:
# Step 2: Define a compute_metrics function
def compute_metrics(eval_pred):
    # Get the true labels and predicted probabilities
    probabilities, labels = eval_pred
    output = np.argmax(probabilities, axis = 1)

    # TODO: compute accuracy between predictions & true labels
    return {'accuracy': sum([o == r for o, r in zip(output, labels)]) / len(labels)}

In [31]:
grader.grade(test_case_id = 'test_compute_metrics', answer = compute_metrics)

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### **Step 3: Initialize the `Trainer` and Specify `TrainingArguments`**

Here we specify various arguments to the trainer. We choose to evaluate and log every `epoch` (i.e. every pass through the full dataset), we specify the output directory for checkpoints, the learning rate (`5e-05`), the number of epochs (`3`), and the size of batches used for stochastic gradient descent (`8`). In addition, we specify full_determinism for the sake of grading consistency.

In the Trainer we specify our model as the BERT model we loaded earlier, we pass in our argument and filtered datasets, and finally we pass in our `compute_metrics` function.

In [32]:
# Step 3: Specify the TrainingArguments and Initialize the Trainer (Do not change!)
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    output_dir="yelp-training",
    learning_rate=5e-05,
    num_train_epochs=3.0,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    full_determinism=True
)

trainer = Trainer(
    model=classification_head_bert,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=compute_metrics,
)

### **Time to Train!**

Once all the above cells have finished running it is time to train the model. On GPU this should only take about 5 minutes. You can check if you're using the GPU by going to `Runtime` -> `Change Runtime Type` and making sure that `Hardware Accelerator` is set to `GPU`

In [33]:
# Train the model!
trainer.train()
trainer.save_model('./yelpBERT')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.592100,1.354960,0.379000
2,1.178700,1.027291,0.566000
3,0.811300,1.015227,0.572000


## Sanity-Checking our trained model

In order to run our trained model we need to load it into a `text-classification` pipeline. A quick sanity check should show us that our model outputs sensible results

In [34]:
yelpBERT = pipeline('text-classification', model='./yelpBERT', tokenizer=tokenizer, device=device_id)

In [35]:
# Output sentiment for reviews (LABEL_4 = 5 stars, LABEL_0 = 1 star)
yelpBERT(["This place was amazing!",
          "This place was good",
          "This place was fine, there were good and bad parts.",
          "This place was pretty bad",
          "This place was awful"])

[{'label': 'LABEL_4', 'score': 0.765678882598877},
 {'label': 'LABEL_3', 'score': 0.4837232828140259},
 {'label': 'LABEL_2', 'score': 0.543403148651123},
 {'label': 'LABEL_1', 'score': 0.6713369488716125},
 {'label': 'LABEL_1', 'score': 0.44839540123939514}]

## Evaluating the model

In order to get the official evaluation results we will be using the [Evaluator](https://huggingface.co/docs/evaluate/v0.4.0/en/package_reference/evaluator_classes#evaluator) pipeline for a standardized evaluation environment. Please run the following code to get the accuracy of your model on the Yelp dataset. We will be grading your model's accuracy. Your model should be at least higher than 55% to recieve full credit for this section.

In [36]:
task_evaluator = evaluator("text-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline=yelpBERT,
    data=eval_data,
    metric=evaluate.load("accuracy"),
    label_mapping={"LABEL_0": 0, "LABEL_1": 1, "LABEL_2": 2, "LABEL_3": 3, "LABEL_4": 4}
)

print(eval_results)

{'accuracy': 0.572, 'total_time_in_seconds': 588.412873992, 'samples_per_second': 1.6994869490459108, 'latency_in_seconds': 0.5884128739919999}


In [37]:
grader.grade(test_case_id = 'test_yelp_accuracy', answer = eval_results)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 3: Training BERT for Natural Language Inference (25 points)

In this section you will train BERT to perform a new task -- Natural Language Inference (NLI). NLI is the task of taking two pieces of text (a "premise" and "hypothesis") and determining whether or not the premise is entailed from the hypothesis.


> Natural Language Inference is a task of determining whether the given “hypothesis” and “premise” logically follow (entailment) or unfollow (contradiction) or are undetermined (neutral) to each other. ~ [Oleh Loksyn](https://towardsdatascience.com/natural-language-inference-an-overview-57c0eecf6517)

This can be formulated as a sequence classification task where, given both sequences, we predict one of three labels

(0 = contradiction, 1 = neutral, 2 = entailment)

### **Applying BERT to multiple sequences**

In order to encode multiple sequences of non-contiguous text with BERT we use the `[SEP]` token. This token is a special token (similar to `[MASK]`) that indicates to the model that the sequences on either side of the token are distinct. With the `[SEP]` token we're able to concatenate the premise and hypothesis together into one sequence allowing us to use a standard classification head.

### **Loading BERT**
Your first task is to load in BERT with a classification head. This should be identical to the loading code from Section 2 save for a different number of labels

In [38]:
# TODO: Load in the bert-base-cased model with a classification head (remember the right number of labels!)
classification_head_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

### **The MNLI Dataset**
For this task we'll be using the [Multi-Genre NLI](https://huggingface.co/datasets/multi_nli) dataset. This dataset was collected by asking crowd workers to annotate 433,000 sentence pairs from various genres (Fiction, Travel, Telephone, Letters, Government) for their textual entailment information.

In [39]:
mnli = load_dataset("multi_nli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
mnli.column_names

{'train': ['promptID',
  'pairID',
  'premise',
  'premise_binary_parse',
  'premise_parse',
  'hypothesis',
  'hypothesis_binary_parse',
  'hypothesis_parse',
  'genre',
  'label'],
 'validation_matched': ['promptID',
  'pairID',
  'premise',
  'premise_binary_parse',
  'premise_parse',
  'hypothesis',
  'hypothesis_binary_parse',
  'hypothesis_parse',
  'genre',
  'label'],
 'validation_mismatched': ['promptID',
  'pairID',
  'premise',
  'premise_binary_parse',
  'premise_parse',
  'hypothesis',
  'hypothesis_binary_parse',
  'hypothesis_parse',
  'genre',
  'label']}

### **Tokenize and Process the Dataset** (15 points)

In this section you must implement the three processing functions `is_labeled`, `concatenate` and `tokenize_function` each worth 5 points. These should be fairly similar to the processing functions we've been dealing with so far.

In [41]:
def is_labeled(row):
  # TODO: Filter out all examples that do not have an entailment label (i.e. label is -1)
  if row['label'] == -1:
    return False
  return True

mnli = mnli.filter(is_labeled)

Filter:   0%|          | 0/392702 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9815 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [42]:
grader.grade(test_case_id = 'test_is_labeled', answer = is_labeled)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [43]:
def concatenate(row):
  # TODO:　Concatenate the text of the "premise" column to the "hypothesis" column
  #        using the [SEP] token. Must be in the order "<premise> [SEP] <hypothesis>"
  #        Add the output as a new column "concat" to the dataset
  row['concat'] = row['premise'] + ' [SEP] ' + row['hypothesis']
  return row

mnli = mnli.map(concatenate)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [44]:
grader.grade(test_case_id = 'test_concatenate', answer = concatenate)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [45]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(row):
  # TODO: tokenize the concatenated data (make sure to pad and truncate to max length!)
  tokens = tokenizer(row['concat'], max_length = tokenizer.model_max_length, padding = 'max_length', truncation = True)
  return {'input_ids': tokens['input_ids'], 'token_type_ids': tokens['token_type_ids'], 'attention_mask': tokens['attention_mask']}

# Randomly select 1000 examples from train and validation and process them (Do not change!)
train_mnli = mnli["train"].shuffle(seed=42).select(range(1000)).map(tokenize_function)
eval_mnli = mnli["validation_matched"].shuffle(seed=42).select(range(1000)).map(tokenize_function)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [95]:
train_mnli = mnli["train"].shuffle(seed=42).select(range(10)).map(tokenize_function)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [96]:
train_mnli

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'concat', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10
})

In [46]:
grader.grade(test_case_id = 'test_tokenize_function', answer = train_mnli[:500])

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### **Define the `compute_metrics` function** (5 points)
For this section you'll define the compute_metrics function. We'll be using accuracy again in this section. Feel free to reference your implementation from the previous section to help you out.

In [47]:
def compute_metrics(eval_pred):
    # Get the true labels and predicted probabilities
    probabilities, labels = eval_pred
    output = np.argmax(probabilities, axis = 1)

    # TODO: compute accuracy between predictions & true labels
    return {'accuracy': sum([o == r for o, r in zip(output, labels)]) / len(labels)}

In [48]:
grader.grade(test_case_id = 'test_mnli_compute_metrics', answer = compute_metrics)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### **Initialize the `TrainingArguments` and `Trainer`**

In [49]:
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    output_dir="mnli-training",
    learning_rate=5e-05,
    num_train_epochs=3.0,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    full_determinism=True
)

trainer = Trainer(
    model=classification_head_bert,
    args=training_args,
    train_dataset=train_mnli,
    eval_dataset=eval_mnli,
    compute_metrics=compute_metrics,
)

### **Train the model**

In [50]:
trainer.train()
trainer.save_model('./mnliBERT')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.102300,1.024936,0.476000
2,0.845000,0.970660,0.542000
3,0.458400,1.215416,0.567000


### **Sanity Check**

Once again, if trained properly, a quick sanity check should provide us with sensible results

In [51]:
mnliBERT = pipeline('text-classification', model='./mnliBERT', tokenizer=tokenizer, device=device_id)

In [52]:
# Output entailment (LABEL_0 = entailment, LABEL_1 = neutral, LABEL_2 = contradiction)
mnliBERT(["I just graduated college. [SEP] The person is a college graduate.",
          "My name is John. [SEP] The sky is blue.",
          "Thank you so much for your help. [SEP] The person was not helped."])

[{'label': 'LABEL_0', 'score': 0.6496190428733826},
 {'label': 'LABEL_1', 'score': 0.9621369242668152},
 {'label': 'LABEL_2', 'score': 0.911557137966156}]

### **Evaluate the Model**

Please run the following code to evaluate your model on the MNLI dataset. Your model should be at least 55% accurate to receive full credit.

In [53]:
task_evaluator = evaluator("text-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline=mnliBERT,
    data=eval_mnli,
    input_column="concat",
    label_column="label",
    metric=evaluate.load("accuracy"),
    label_mapping={"LABEL_0": 0, "LABEL_1": 1, "LABEL_2": 2}
)

print(eval_results)

{'accuracy': 0.567, 'total_time_in_seconds': 162.6328245090001, 'samples_per_second': 6.14882022137333, 'latency_in_seconds': 0.1626328245090001}


In [54]:
grader.grade(test_case_id = 'test_mnli_accuracy', answer = eval_results)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 4: Training BERT for Named-Entity Recognition (40 points)

In this final section you will train BERT to perform [Named Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition) (NER).

NER is the task of tagging and identifying named entities (e.g. People, Organizations, Locations) in text. For example the sentence "Only France and Britain backed Fischler's proposal." would be tagged as

> Only (France, LOC) and (Britain, LOC) backed (Fischler's, PER) proposal.

NER is typically formulated as a token classification task. Given a sequence of tokens we assign a class to each token. Tokens that are not named entities get given the null tag `O` and tokens that are named entities get one of four classes (`ORG`, `PER`, `LOC`, or `MISC`). Thus the sentence from earlier would be given the labels

>(Only, O) (France, LOC) (and, O) (Britain, LOC) (backed, O) (Fischler's, PER) (proposal, O)

Giving every token in the sequence a label allows us to use a token classification head for this task.

### **Beginning-Inside-Outside (BIO) tags**

One downside to the tagging scheme described is that for two consecutive tokens with the same tag, we can't tell whether or not they are the same entity. For example

> ("25-1", 0), ("Barcelona", ORG), ("Real", ORG), ("Madrid", ORG)

We know that [Real Madrid](https://en.wikipedia.org/wiki/Real_Madrid_CF) and [Barcelona](https://en.wikipedia.org/wiki/FC_Barcelona) are distinct entities but our tag system combines them together. Thus we must distinguish between tags that *begin* an entity (`B-ORG`) and those that are *inside* an entity (`I-ORG`). With this new system we get

> ("25-1", O), ("Barcelona", B-ORG), ("Real", B-ORG), ("Madrid", I-ORG)

Which allows us to distinguish between the two consecutive entities.

### **Training BERT + Token Classification Head**

In order to train BERT for NER we need to use a token classification head. Token classification heads are dense linear layers that output a probability distribution over $n$ classes *for each token* in the sequence. This is equivalent to having $L_{context}$ different classification heads, each trained to predict an output at a specific index. The total size of a token classification head is ($L_{context} * D_{enc}$ x $n * L_{context}$).

### **Load the Model** (5 points)
Your first task is to load BERT with a token classification head. Take care that the `num_labels` and `id2label` parameters are set correctly.

HINT: Refer back to Section 2's discussion of model heads to find mention (and documentation) for the token classification head class.

In [55]:
label_names = {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8:'I-MISC'}
# TODO: Load the model with the right num_labels and set id2label to be label_names
token_classification_head_bert = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=9, id2label = label_names)

In [56]:
attributes = copy.deepcopy(vars(token_classification_head_bert))
attributes['config'] = attributes['config'].__dict__
del attributes['_modules']
grader.grade(test_case_id = 'test_load_bert', answer = attributes)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### **The CoNLL Dataset**

We will use the [CoNLL](https://huggingface.co/datasets/conll2003) dataset as our training data for NER. This dataset consists of 20,000 sentences from Reuters news articles that were manually annotated by participants for their named entities. The tags used are the four classes from earlier (`ORG`, `PER`, `LOC`, or `MISC`) with both `B-` and `I-` variants as well as a null tag `O`.

In [57]:
conll = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [58]:
conll.column_names

{'train': ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
 'validation': ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
 'test': ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']}

### **Token-tag mismatch**
When processing data for a tagging task, we need to ensure that the tags properly match up to all the correct tokens. BERT's tokenizer is a [Byte-Pair Encoding](https://en.wikipedia.org/wiki/Byte_pair_encoding) (BPE) tokenizer that tends to split up words into sub-word tokens. For example, take the sentence from earlier
> "Only France and Britain backed Fischler's proposal"

We can see below that the tokenization from CoNLL doesn't match the BERT tokenizer. This creates a problem when processing the tags, as we need to keep the correspondence between tags and tokens. It will be up to you to fix this

In [59]:
print(conll['train'][12]['ner_tags'])
print(tokenizer.convert_ids_to_tokens(tokenizer.encode("Only France and Britain backed Fischler's proposal"))[1:-1])

[0, 5, 0, 5, 0, 1, 0, 0, 0]
['Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal']


In [60]:
print(conll['train'][12]['tokens'])
print(conll['train'][12]['ner_tags'])

['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
[0, 5, 0, 5, 0, 1, 0, 0, 0]


In [61]:


encoded_input = tokenizer(conll['train'][12]['tokens'],  is_split_into_words=True, max_length = tokenizer.model_max_length, padding = 'max_length', truncation = True)

In [62]:
words = conll['train'][12]['tokens']
word_labels = conll['train'][12]['ner_tags']
tokens = []
labels = []
for word, label in zip(words, word_labels):
     # tokenize the word
     word_tokens = tokenizer.tokenize(word)
     # propagate label to all tokens of the given word
     tokens.extend(word_tokens)
     labels.extend([label] * len(word_tokens))

print(tokens)
print(labels)

final_labels = [-100]
final_tokens = tokenizer.convert_ids_to_tokens(tokenizer(words, max_length = tokenizer.model_max_length, padding = 'max_length', truncation = True,  is_split_into_words=True)['input_ids'])
if len(tokens) < tokenizer.model_max_length -1:
  final_labels.extend(labels)
  final_labels.extend([-100]*(tokenizer.model_max_length - len(final_labels)))
else:
  finals_labels.extend(labels[:tokenizer.model_max_length -1])

print(final_labels)
print(final_tokens)

['Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.']
[0, 5, 0, 5, 0, 1, 1, 1, 0, 0, 0, 0]
[-100, 0, 5, 0, 5, 0, 1, 1, 1, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

### **Redistributing Tags to Sub-word Tokens** (10 points)

Your job is to implement the `tokenize_and_tag` function. This function should do three things
1. Retokenize the tokens from CoNLL using the BERT tokenizer and add the outputs (`input_ids`, `attention_mask`, and `token_type_ids`) as new columns in the dataset.
2. Redistribute the NER tags with respect to the newly tokenized sequence and add the tags as a new `labels` column to the dataset
3. Pad and truncate both the tokens and tags to be length `tokenizer.model_max_length` (512)

If a token is broken up into multiple sub-tokens then all sub-tokens should be given the same NER tag as the original token.

Some example inputs and outputs are as follows (Reminder to consult the `label_names` dictionary for the mapping from number to NER tag):
```
-- Example #1 --
Inputs:
"tokens": ['Israel', 'approves', 'Arafat', "'s", 'flight', 'to', 'West', 'Bank', '.']
"ner_tags": [5, 0, 1, 0, 0, 0, 5, 6, 0]

Outputs:
"bert_tokens": ['[CLS]', 'Israel', 'approve', '##s', 'Ara', '##fa', '##t', "'", 's', 'flight', 'to', 'West', 'Bank', '.', '[SEP]', '[PAD]', '[PAD]', ...]
"input_ids": [101, 3103, 14942, 1116, 25692, 8057, 1204, 112, 188, 3043, 1106, 1537, 2950, 119, 102, 0, 0, ...]
"labels": [-100, 5, 0, 0, 1, 1, 1, 0, 0, 0, 0, 5, 6, 0, -100, -100, -100, ...]

-- Example #2 --
Inputs:
"tokens": ['66', 'Paul', 'Goydos', ',', 'Billy', 'Mayfair', ',', 'Hidemichi', 'Tanaka', '(', 'Japan', ')']
"ner_tags": [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 5, 0]

Outputs:
"bert_tokens": ['[CLS]', '66', 'Paul', 'Go', '##yd', '##os', ',', 'Billy', 'May', '##fair', ',', 'Hi', '##de', '##mic', '##hi', 'Tanaka', '(', 'Japan', ')', '[SEP]', '[PAD]', '[PAD]', ...]
"input_ids": [101, 5046, 1795, 3414, 19429, 2155, 117, 4224, 1318, 19803, 117, 8790, 2007, 7257, 3031, 24128, 113, 1999, 114, 102, 0, 0, ...]
"labels": [-100, 0, 1, 2, 2, 2, 0, 1, 2, 2, 0, 1, 1, 1, 1, 2, 0, 5, 0, -100, -100, -100, ...]
```
**NOTE:** Make sure to use the label `-100` for all special tokens (`[CLS]`, `[SEP]`, `[PAD]`). Using this label will ensure that special tokens will be ignored by PyTorch and will not contribute to the loss calculation.

In [97]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_and_tag(row):
  # TODO: Redistribute and tokenize the CoNLL dataset using the BERT tokenizer
  words = row['tokens']
  word_labels = row['ner_tags']
  tokens = []
  labels = []

  # redistribute the tokens from CoNLL dataset
  for word, label in zip(words, word_labels):
      # tokenize the word
      word_tokens = tokenizer.tokenize(word)
      # propagate label to all tokens of the given word
      tokens.extend(word_tokens)
      labels.extend([label] * len(word_tokens))

  # add padding and keep fixed length using the BERT tokenizer
  final_labels = [-100]
  bert_tokenizer = tokenizer(words, max_length = tokenizer.model_max_length, padding = 'max_length', truncation = True, is_split_into_words=True)
  final_tokens = tokenizer.convert_ids_to_tokens(bert_tokenizer['input_ids'])
  if len(tokens) < tokenizer.model_max_length -1:
    final_labels.extend(labels)
    final_labels.extend([-100]*(tokenizer.model_max_length - len(final_labels)))
  else:
    final_labels.extend(labels[:tokenizer.model_max_length -1])

  return {'input_ids': bert_tokenizer['input_ids'], 'token_type_ids': bert_tokenizer['token_type_ids'], \
          'attention_mask': bert_tokenizer['attention_mask'], 'labels': final_labels}

# TODO: randomly select 1000 examples from train and validation and call tokenize_and_tag
train_conll = conll["train"].shuffle(seed=42).select(range(1500)).map(tokenize_and_tag)
eval_conll = conll["validation"].shuffle(seed=42).select(range(1500)).map(tokenize_and_tag)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [64]:
grader.grade(test_case_id = 'test_tokenize_and_tag', answer = conll["train"].select(range(350)).map(tokenize_and_tag)[:])

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

Correct! You earned 20/20 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### **Computing Macro-Averaged F1 Score** (20 points)

We want to compute the accuracy of our model when identifying named entities. However, most sequences consist mostly of `O` tags. This is bad for our evaluation, as it means that predicting a sequence of all `O`s will result in high accuracy despite not actually accomplishing the task.

To solve this problem we will be using the Macro-Averaged F1 Score as our metric. This is the unweighted average of the F1 scores for each individual token class. This is a common metric to use when given an unbalanced multi-class classification task. Feel free to read [this article](https://stephenallwright.com/micro-vs-macro-f1-score/) for a full explanation of Macro-F1 as it will be useful for your implementation.

You are to implement the `calculate_macro_f1` function. This function should:
1. Calculate the full confusion matrix (TP, TN, FP, FN) for each class
2. Compute the precision, recall, and f1 score for each class
3. Macro-Average together the F1 score for all nine classes to get the final score

**NOTE**: Typically for NER a correct entity prediction is considered to be one that exactly matches all `B-` and `I-` tags. However, to make the Macro-F1 calculation easier we are considering all of our nine different tag classes as fully separate.

**TIP**: You can use the [sklearn.metrics.f1_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) function from scikitlearn to check your implementation. To help debug we have provided the handy `debug_macro_f1` function for you to use. This function generates random test cases and compares the output of your implementation to that of scikitlearn's. If the two do not match it prints out the failed example. This should be helpful when debugging.

In [65]:
def calculate_macro_f1(preds, labels):
  # Filter out -100 and convert ids to tags
  label_map = {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8:'I-MISC'}
  true_preds = [
      [(label_map[p], label_map[l]) for (p, l) in zip(pred, label) if l in label_map]
      for pred, label in zip(preds, labels)
  ]

  # TODO: Compute confusion matrices for each class, Compute precision, recall, and f1 for each class
  f1_o = compute_f1(true_preds, 'O')
  f1_bper = compute_f1(true_preds, 'B-PER')
  f1_iper = compute_f1(true_preds, 'I-PER')
  f1_borg = compute_f1(true_preds, 'B-ORG')
  f1_iorg = compute_f1(true_preds, 'I-ORG')
  f1_bloc = compute_f1(true_preds, 'B-LOC')
  f1_iloc = compute_f1(true_preds, 'I-LOC')
  f1_bmisc = compute_f1(true_preds, 'B-MISC')
  f1_imisc = compute_f1(true_preds, 'I-MISC')

  # TODO: Macro-Average them together
  print(f'my f1 {f1_o} {f1_bper} {f1_iper} {f1_borg} {f1_iorg} {f1_bloc} {f1_iloc} {f1_bmisc} {f1_imisc}')

  # filter out the classes never appeared in both label and pred
  f1_lst = [f1_o, f1_bper, f1_iper, f1_borg, f1_iorg, f1_bloc, f1_iloc, f1_bmisc, f1_imisc]
  filtered = [f1 for f1 in f1_lst if f1 > -1]
  macro_f1 = sum(filtered)/len(filtered)

  return {"macro-f1": macro_f1}

In [66]:
def compute_confusion_matrix(preds, class_label):
  '''
  input: true predictions as a list of lists of prediction and label pairs,
        and the class_label(string) that you want to compute confusion matrix for
  output: a dictionary that has confusion matrix (TP, TN, FP, FN) as keys, the count as values
  '''
  matrix = {'tp':0, 'tn':0, 'fp':0, 'fn':0}

  for pred in preds:
    for p, l in pred:
      if l == class_label and p == class_label:
        matrix['tp'] += 1
      elif l == class_label and p != class_label:
        matrix['fn'] += 1
      elif l != class_label and p == class_label:
        matrix['fp'] += 1
      elif l != class_label and p != class_label:
        matrix['tn'] += 1

  return matrix

In [67]:
def compute_f1(preds, class_label):
  '''
  input: true predictions as a list of lists of prediction and label pairs,
        and the class_label(string) that you want to compute confusion matrix for

  A dictionary that has confusion matrix (TP, TN, FP, FN) as keys, the count as values will be computed.

  output: f1 score for this class prediction
  '''
  matrix = {'tp':0, 'tn':0, 'fp':0, 'fn':0}

  for pred in preds:
    for p, l in pred:
      if l == class_label and p == class_label:
        matrix['tp'] += 1
      elif l == class_label and p != class_label:
        matrix['fn'] += 1
      elif l != class_label and p == class_label:
        matrix['fp'] += 1
      elif l != class_label and p != class_label:
        matrix['tn'] += 1

  if matrix['tp'] != 0:
    f1 = 2*matrix['tp']/(2*matrix['tp'] + matrix['fn'] + matrix['fp'])
  elif matrix['fn'] == 0 and matrix['fp'] == 0:
    f1 = -1
  else:
    f1 = 0

  return f1

In [68]:
grader.grade(test_case_id = 'test_macro_f1', answer = calculate_macro_f1)

Correct! You earned 20/20 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [69]:
from sklearn.metrics import f1_score

def debug_macro_f1(num_tests, len_examples):
  # For each random test case
  for i in range(num_tests):
    # Generate two random arrays for the predictions and labels
    rand_preds = np.random.randint(0,9,len_examples).tolist()
    rand_labels = np.random.randint(0,9,len_examples).tolist()

    # Calculate Macro-F1 score with your implementation and scikitlearn
    sklearn_f1 = f1_score(rand_labels, rand_preds, average='macro')
    your_f1 = calculate_macro_f1([rand_preds], [rand_labels])

    # If the two implementations differ, print the example and the scores
    if abs(sklearn_f1 - your_f1['macro-f1']) <= 0.001:
      print('PASS:')
      print(f'preds: {rand_preds}')
      print(f'labels: {rand_labels}')
      print(sklearn_f1, your_f1['macro-f1'])
    else:
      print('FAIL:')
      print(f'preds: {rand_preds}')
      print(f'labels: {rand_labels}')
      print(sklearn_f1, your_f1['macro-f1'])
debug_macro_f1(5, 5)

my f1 -1 -1 0 0 1.0 -1 0 1.0 -1
PASS:
preds: [6, 3, 7, 4, 6]
labels: [2, 6, 7, 4, 3]
0.4 0.4
my f1 -1 0 0 -1 1.0 0 -1 0.6666666666666666 -1
PASS:
preds: [7, 7, 2, 5, 4]
labels: [1, 7, 5, 1, 4]
0.3333333333333333 0.3333333333333333
my f1 0 -1 0 0 0 0 0 -1 1.0
PASS:
preds: [0, 5, 8, 0, 2]
labels: [6, 3, 8, 2, 4]
0.14285714285714285 0.14285714285714285
my f1 -1 0 0 0 0 -1 0 -1 0
PASS:
preds: [2, 6, 4, 8, 6]
labels: [1, 3, 8, 1, 8]
0.0 0.0
my f1 0 0 0 1.0 0 -1 0 1.0 -1
PASS:
preds: [4, 1, 3, 6, 7]
labels: [2, 0, 3, 1, 7]
0.2857142857142857 0.2857142857142857


In [70]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=2)
  return calculate_macro_f1(preds, labels)

### **Training the model**

In [98]:
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    output_dir="conll-training",
    learning_rate=5e-05,
    num_train_epochs=3.0,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    full_determinism=True
)

trainer = Trainer(
    model=token_classification_head_bert,
    args=training_args,
    train_dataset=train_conll,
    eval_dataset=eval_conll,
    compute_metrics=compute_metrics
)

In [99]:
trainer.train()
trainer.save_model('./nerBERT')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Macro-f1
1,0.367200,0.136635,0.814664
2,0.103500,0.105591,0.876130
3,0.043200,0.105704,0.892306


my f1 0.9888205583380882 0.9435454872348076 0.9857464366091523 0.848659793814433 0.7949308755760369 0.8855869242199108 0.6724137931034483 0.690854119425548 0.521415270018622
my f1 0.9915404788088994 0.9530916844349681 0.9876635514018691 0.8906644238856182 0.8693467336683417 0.930962962962963 0.8478802992518704 0.7937411095305832 0.6202783300198808
my f1 0.9929741397553646 0.9576361694553222 0.9887724550898204 0.899556272690601 0.8896797153024911 0.935881627620222 0.8708860759493671 0.8002726653033402 0.6950959488272921


### **Sanity Checking the NER Predictions**
In order to check our model we need to load it into a `token-classification` pipeline.

If your model has been trained correctly then you will see in the following example that it correctly predicts the "Pennsylvania" in "University of Pennsylvania" as an `ORG` but "Philadelphia" as a `LOC`.

In [100]:
nerBERT = pipeline('token-classification', model='./nerBERT', tokenizer=tokenizer, device=device_id)

In [101]:
nerBERT(["Chris Callison-Burch is a professor at the University of Pennsylvania in Philadelphia",
          "Joe Biden is the 46th President of the United States"], aggregation_strategy="simple")

[[{'entity_group': 'PER',
   'score': 0.99800164,
   'word': 'Chris Callison - Burch',
   'start': 0,
   'end': 20},
  {'entity_group': 'ORG',
   'score': 0.858815,
   'word': 'University of Pennsylvania',
   'start': 43,
   'end': 69},
  {'entity_group': 'LOC',
   'score': 0.9902215,
   'word': 'Philadelphia',
   'start': 73,
   'end': 85}],
 [{'entity_group': 'PER',
   'score': 0.9979672,
   'word': 'Joe Biden',
   'start': 0,
   'end': 9},
  {'entity_group': 'LOC',
   'score': 0.8608484,
   'word': 'United States',
   'start': 39,
   'end': 52}]]

### **Evaluating NER with SeqEval**

The official library for evaluating a model on the CoNLL task is [SeqEval](https://github.com/chakki-works/seqeval). We'll be downloading it and using it to officially score our model. The SeqEval `overall_f1` is identical to your Macro-F1 implementation with the slight caveat that it uses the traditional NER definition of a correct tag prediction (i.e. one that exactly matches all `B-` and `I-` tags). Thus, don't worry if the official result doesn't match your macro-f1 implementation.

For this task, in order to receive full credit, your implementation must get a Macro-F1 score of at least 0.82.

In [102]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=0d792304fe50e1e402e11dc93bd908c217a6532cc8d31c637edd7fed6d53b1d1
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [103]:
task_evaluator = evaluator("token-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline=nerBERT,
    data=eval_conll,
    input_column="tokens",
    label_column="ner_tags",
    tokenizer=tokenizer,
    metric=evaluate.load("seqeval")
)

for key in eval_results:
  print(f"{key}: {eval_results[key]}")

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

LOC: {'precision': 0.9424882629107981, 'recall': 0.9251152073732719, 'f1': 0.9337209302325581, 'number': 868}
MISC: {'precision': 0.7516059957173448, 'recall': 0.8013698630136986, 'f1': 0.7756906077348066, 'number': 438}
ORG: {'precision': 0.8353658536585366, 'recall': 0.898360655737705, 'f1': 0.8657187993680885, 'number': 610}
PER: {'precision': 0.9639751552795031, 'recall': 0.9663760896637609, 'f1': 0.9651741293532339, 'number': 803}
overall_precision: 0.8913669064748202
overall_recall: 0.9113644722324384
overall_f1: 0.9012547735951992
overall_accuracy: 0.9845620937393089
total_time_in_seconds: 199.19555562699952
samples_per_second: 7.53028849101835
latency_in_seconds: 0.13279703708466634


In [104]:
grader.grade(test_case_id = 'test_ner_score', answer = eval_results)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### Congratulation on finishing the homework! Here are the deliverables you need to submit to GradeScope
- This notebook and py file: rename to `homework7.ipynb` and `homework7.py`. You can download the notebook and py file by going to the top-left corner of this webpage, `File -> Download -> Download .ipynb/.py`